In [24]:
import os

import pandas as pd

In [25]:
def read_pacing_metrics(path: str, subdir: str = "summary") -> pd.DataFrame:
    metrics_df = pd.read_csv(f"{path}/{subdir}/metrics.csv")
    config_df = pd.read_csv(f"{path}/{subdir}/config.csv")
    df = pd.merge(metrics_df, config_df, on="run_name")
    return df


def select_baseline_runs(df: pd.DataFrame, dataset: str) -> pd.DataFrame:
    df = df[~df["curriculum"].notnull()]
    df = df[df["seed"] == 1]
    return df


def create_baseline_performance(dataset: str) -> pd.DataFrame:
    df = read_pacing_metrics(f"results/{dataset}")
    df = select_baseline_runs(df, dataset)
    df = (
        df.sort_values("test_accuracy", ascending=False)
        .groupby("model")
        .head(1)
        .reset_index(drop=True)
    )
    return df[["model", "optimizer", "learning_rate", "test_accuracy"]]

In [26]:
cifar_df = create_baseline_performance("cifar")
dcase_df = create_baseline_performance("dcase")
os.makedirs("results/tables", exist_ok=True)
cifar_df.to_csv("results/tables/4_a_baselines_cifar.csv", index=False)
dcase_df.to_csv("results/tables/4_a_baselines_dcase.csv", index=False)

In [27]:
cifar_df.round(3)

,model,optimizer,learning_rate,test_accuracy
0,ResNet-50-T,SAM-SGD-M9,0.001,0.949
1,EfficientNet-B4-T,SAM-SGD-M9,0.001,0.945
2,EfficientNet-B0-T,SAM-SGD-M9,0.001,0.936
3,EfficientNet-B4,Adam,0.001,0.848
4,EfficientNet-B0,Adam,0.001,0.835
5,ResNet-50,Adam,0.001,0.813


In [28]:
dcase_df.round(3)

,model,optimizer,learning_rate,test_accuracy
0,Cnn14-32k-T,Adam,0.000,0.678
1,Cnn10-32k-T,SAM-SGD-M9,0.010,0.653
2,EfficientNet-B0-T,SAM-SGD-M9,0.010,0.644
3,Cnn10,SAM-SGD-M9,0.001,0.609
4,Cnn14,SAM-SGD-M9,0.001,0.595
5,EfficientNet-B0,SAM-SGD-M9,0.010,0.583
